In [1]:
import os
import gzip
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def read_nk(filepath, wavetype, filetype='gz'):
    '''
    Nlhoncohden 파일을 읽기 위한 함수
    filepath : waveform filepath
    wavetype : ECGII, ART1(A-line Blood pressure),RESPIMP(Respiratory Impedance),SPO2IRAC(PPG),CVP(Central Venous pressure)
    filetype : gz or csv  (초창기 csv로 저장된 파일을 읽기위함)
    
    Return
    biosignal array(numpy array)
    '''
    if filetype == 'gz':
        with gzip.open(filepath, 'rt') as f:
            content = f.read()
            if wavetype == 'ECGII':
                tmp = np.int32((np.array(content.split(','))[:-1]).reshape(-1))
                tmp = tmp[::2]  #sampling rate 조절하는 부분
                return tmp

            elif wavetype == 'ART1' or wavetype == 'RESPIMP' or wavetype == 'SPO2IRAC' or wavetype == 'CVP':
                tmp = np.int32((np.array(content.split(','))[:-1]).reshape(-1))
                return tmp
    elif filetype == 'csv':
        tmp = pd.read_csv(filepath, header=None).iloc[:, :-1]
        if wavetype == 'ECGII':
            tmp = np.array(tmp).reshape(-1)
            tmp = tmp[::2] #sampling rate 조절하는 부분
            return tmp
        tmp = np.array(tmp).reshape(-1)
        return tmp
def read_ph(filepath, wavetype, filetype=None):
    tmp = zipfile.ZipFile(filepath)
    if wavetype == 'ECGII':
        tmp = pd.read_csv(tmp.open(tmp.namelist()[0]), header=None)[list(range(5000))]
        tmp = np.array(tmp).reshape(-1)
        tmp = tmp[::4]
        return tmp  ## 여기 스케일 조정해야돼 스케일 이슈
    elif wavetype == 'ART1' or wavetype == 'SPO2IRAC':
        tmp = pd.read_csv(tmp.open(tmp.namelist()[0]), header=None)[list(range(1250))]
        tmp = np.array(tmp).reshape(-1)
        return tmp
    elif wavetype == 'RESPIMP':
        tmp = pd.read_csv(tmp.open(tmp.namelist()[0]), header=None)[list(range(625))]
        tmp = np.array(tmp).reshape(-1)
        tmp = np.interp(np.arange(0, len(tmp), 0.5), np.arange(0, len(tmp)), tmp)
        return tmp
    return None

In [3]:
import pandas as pd

In [4]:
NCUA=pd.read_csv('NCUA.csv')
NCUB=pd.read_csv('NCUB.csv')

In [5]:
NCUA_ID=NCUA['patient_id']
NCUB_ID=NCUB['patient_id']

In [7]:
NCUA_ID=list(NCUA_ID)

In [8]:
NCUB_ID=list(NCUB_ID)

In [9]:
NCU_ID=NCUA_ID+NCUB_ID

In [10]:
#len(NCU_ID)

1275

In [12]:
GZIP_NCU = []
for i in range(len(NCU_ID)):
    if NCU_ID[i] in os.listdir('/mount/GZIP/'):
        GZIP_NCU.append(NCU_ID[i])

In [13]:
GZIP_dict = {}
#GZIP_dict['NCU_ID'] = {}
for i in range(len(GZIP_NCU)):
    GZIP_dict[GZIP_NCU[i]] = {}
    GZIP_dict[GZIP_NCU[i]]['date']=os.listdir('/mount/GZIP/'+GZIP_NCU[i])

In [14]:
#len(GZIP_NCU)

904

In [15]:
GZIP_P = []
for i in range(len(NCU_ID)):
    if NCU_ID[i] in os.listdir('/mount/GZIP_P/'):
        GZIP_P.append(NCU_ID[i])

In [16]:
#len(GZIP_P)

47

In [17]:
GZIP_P_dict = {}
for i in range(len(GZIP_P)):

    GZIP_P_dict[GZIP_P[i]] = {}
    GZIP_P_dict[GZIP_P[i]]['date']=os.listdir('/mount/GZIP_P/'+GZIP_P[i])

In [18]:
GZIP_G = []
for i in range(len(NCU_ID)):
    if NCU_ID[i] in os.listdir('/mount/GZIP_G/'):
        GZIP_G.append(NCU_ID[i])

In [19]:
GZIP_G_dict = {}
for i in range(len(GZIP_P)):

    GZIP_G_dict[GZIP_G[i]]= {}
    GZIP_G_dict[GZIP_G[i]]['date']=os.listdir('/mount/GZIP_G/'+GZIP_G[i])

In [ ]:
import pickle
with open('GZIP_G_dict.pickle', 'wb') as f:
    pickle.dump(GZIP_G_dict, f)
with open('GZIP_dict.pickle', 'wb') as f:
    pickle.dump(GZIP_dict, f)
with open('GZIP_P_dict.pickle', 'wb') as f:
    pickle.dump(GZIP_P_dict, f)

In [ ]:
fig = plt.figure()## 그래프 객체 생성
for w in range(5,20):
    ECGII_list = []
    RESPIMP_list = []
    SPO2IRAC_list = []
    if GZIP_NCU[w] in os.listdir('/mount/GZIP'):
        date_list = os.listdir('/mount/GZIP/'+(GZIP_NCU[w]))
        try:
            os.mkdir('graph'+'/'+GZIP_NCU[w])
        except:
            pass
        for q in range(len(date_list)):
            signal_list=os.listdir('/mount/GZIP'+'/'+GZIP_NCU[w]+'/'+date_list[q])
            try:
                os.mkdir('graph'+'/'+GZIP_NCU[w]+'/'+date_list[q])
            except:
                pass
            for i in range(int(len(signal_list))):
                if signal_list[i].split('_')[-1] == 'ECGII.gz':
                    ECGII_list.append(signal_list[i])
                elif signal_list[i].split('_')[-1] == 'RESPIMP.gz':
                    RESPIMP_list.append(signal_list[i])
                elif signal_list[i].split('_')[-1] == 'SPO2IRAC.gz':
                    SPO2IRAC_list.append(signal_list[i])            
            for e in range(len(ECGII_list)):
                tmp = ECGII_list[e].split('_')[0] + '_' +ECGII_list[e].split('_')[1]
                try:
                    ECGII_ = read_nk('/mount/GZIP/'+'/'+GZIP_NCU[w]+'/'+date_list[q]+'/'+tmp+'_'+'ECGII.gz','ECGII')
                    RESPIMP_ = read_nk('/mount/GZIP/'+'/'+GZIP_NCU[w]+'/'+date_list[q]+'/'+tmp+'_'+'RESPIMP.gz','RESPIMP')
                    SPO2IRAC_ = read_nk('/mount/GZIP/'+'/'+GZIP_NCU[w]+'/'+date_list[q]+'/'+tmp+'_'+'SPO2IRAC.gz','SPO2IRAC')
                    plt.rcParams['figure.figsize'] = (28,14)## 그래프 크기 지정
                    
                    ax1 = fig.add_subplot(3,1,1) ## 3가지 데이터 부분 그래프 생성
                    ax1.plot(np.arange(0, 600, 600/len(ECGII_)),ECGII_,linewidth=0.1,color='red')##10분간의 그래프
                    #plt.ylabel('Sec')
                    plt.title('RESPIMP')
                    ax2 = fig.add_subplot(3,1,2)
                    plt.title('SPO2')
                    ax2.plot(np.arange(0, 600, 600/len(RESPIMP_)),RESPIMP_,linewidth=0.1,color='green')
                    ax3 = fig.add_subplot(3,1,3)
                    plt.title('ECG')
                    ax3.plot(np.arange(0, 600, 600/len(SPO2IRAC_)),SPO2IRAC_,linewidth=0.1,color='blue')
                    fig.savefig('graph/'+GZIP_NCU[w]+'/'+date_list[q]+'/'+tmp+'.png') 
                    fig.clear()
                except:
                    pass

In [ ]:
PH

'3ICUA'
'3ICUB'
'EICU'
'MICU'
'TICUA'
'TOR'

In [ ]:
NK

'NULL'
'BED'
'ER_ISO'
'ER_PED'
'ER_RES'
'ICU'
'NCU B'
'TICUB'
'TICUC'

In [ ]:
GE

'091S'
'3CCU'
'5ICU'
'CTC'
'MPA'
'NCUA'
'OR'